In [ ]:
def parse_sig(buf: bytes):
    m = re.search(rb"Signature \(in hex\):\s*([0-9a-fA-F]+)", buf)
    if not m:
        raise ValueError("Could not parse signature")
    return m.group(1).decode()

def build_enc_msg(K_server, K_flag, idx=16):
    nonce = urandom(12)

    S1_blocks, EK1_J0 = gcm_keystream_blocks(K_server, nonce, 2)
    S2_blocks, EK2_J0 = gcm_keystream_blocks(K_flag, nonce, 2)
    S1 = b"".join(S1_blocks)
    S2 = b"".join(S2_blocks)
    Delta = bytes(a ^ b for a, b in zip(S1, S2))

    p2_block0 = b"admin = True" + b" "*(16 - len(b"admin = True"))
    p1_block0 = bytes(a ^ b for a, b in zip(p2_block0, Delta[:16]))
    if b"admin = True" in p1_block0:
        p2_block0 = b"admin = True" + b"\x00"*(16 - len(b"admin = True"))
        p1_block0 = bytes(a ^ b for a, b in zip(p2_block0, Delta[:16]))

    p1_block1 = b"admin = False" + b" "*(16 - len(b"admin = False"))
    p2_block1 = bytes(a ^ b for a, b in zip(p1_block1, Delta[16:32]))
    if b"admin = False" in p2_block1:
        p1_block1 = b"admin = False" + b"\x00"*(16 - len(b"admin = False"))
        p2_block1 = bytes(a ^ b for a, b in zip(p1_block1, Delta[16:32]))

    P1 = p1_block0 + p1_block1
    C0 = bytes(m ^ k for m, k in zip(P1, S1))

    C, tag = make_dual_valid(K_server, K_flag, nonce, C0)

    m1 = AES.new(K_server, AES.MODE_GCM, nonce=nonce).decrypt_and_verify(C, tag)
    m2 = AES.new(K_flag, AES.MODE_GCM, nonce=nonce).decrypt_and_verify(C, tag)
    assert (b'admin = False' in m1[idx:idx+16]) and (b'admin = True' not in m1)
    assert (b'admin = True' in m2[:idx]) and (b'admin = False' not in m2)

    enc_msg = nonce + C + tag
    return enc_msg

def main():
    print("[*] Connecting to {}:{}...".format(HOST, PORT))
    s = socket.create_connection((HOST, PORT))
    buf = recv_until(s, PROMPT)
    sys.stdout.write(buf.decode(errors="ignore"))

    print("[*] Rotating Server (idx -> 16)...")
    sendline(s, "1")
    buf = recv_until(s, PROMPT)
    sys.stdout.write(buf.decode(errors="ignore"))
    K_server_a = parse_key(buf)
    print(f"[+] Server key 1: {K_server_a.hex()}")

    print("[*] Rotating FlagKeeper (idx -> 0)...")
    sendline(s, "2")
    buf = recv_until(s, PROMPT)
    sys.stdout.write(buf.decode(errors="ignore"))
    K_flag = parse_key(buf)
    print(f"[+] FlagKeeper key: {K_flag.hex()}")

    print("[*] Rotating Server again (idx -> 16)...")
    sendline(s, "1")
    buf = recv_until(s, PROMPT)
    sys.stdout.write(buf.decode(errors="ignore"))
    K_server = parse_key(buf)
    print(f"[+] Server key 2: {K_server.hex()}")

    print("[*] Building dual-valid ciphertext...")
    enc_msg = build_enc_msg(K_server, K_flag, idx=16)
    enc_hex = enc_msg.hex()
    print(f"[+] Ciphertext: {enc_hex[:60]}...")

    print("[*] Requesting signature...")
    sendline(s, "3")
    recv_until(s, b"encrypted message (in hex):")
    sendline(s, enc_hex)
    buf = recv_until(s, PROMPT)
    sys.stdout.write(buf.decode(errors="ignore"))
    sig_hex = parse_sig(buf)
    print(f"[+] Got signature: {sig_hex[:60]}...")

    print("[*] Getting flag...")
    sendline(s, "4")
    recv_until(s, b"encrypted message (in hex):")
    sendline(s, enc_hex)
    recv_until(s, b"signature (in hex):")
    sendline(s, sig_hex)
    buf = recv_until(s, PROMPT)
    out = buf.decode(errors="ignore")
    sys.stdout.write(out)
    m = re.search(r"Flag:\s*(.*)", out)
    if m:
        print("\n[+] 🎉 " + m.group(0))
    s.close()

if _name_ == "_main_":
    main()

In [ ]:
from sys import exit
from Crypto.Util.number import bytes_to_long, inverse, long_to_bytes
import random
import string
from pwn import *
CHARSET = string.ascii_uppercase + string.ascii_lowercase + string.digits


def main():
    HOST = "crypto1.cscv.vn"
    PORT = 1337
    PROMPT = b"choice: "
    
    print(f"[*] Connecting to {HOST}:{PORT}...")
    io = remote(HOST, PORT)
    buf = io.recvuntil(PROMPT)
    io.sendline(b"1")
    io.recvuntil("current Server key:  ")
    Sk = io.recvline().strip()
    
    print(f"[+] Server key: {Sk.decode()}")    
main()


[*] Connecting to crypto1.cscv.vn:1337...
[x] Opening connection to crypto1.cscv.vn on port 1337
[x] Opening connection to crypto1.cscv.vn on port 1337: Trying 42.96.60.48
[+] Opening connection to crypto1.cscv.vn on port 1337: Done
629638cf7a399b4bfc819b516c5451c2


/tmp/ipykernel_209570/1867304063.py:18: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  io.recvuntil("current Server key:  ")
